In [1]:
import os
import pandas as pd
from datasets import load_dataset

/opt/miniconda3/envs/olga_scipts_experiment/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
openai_key = os.environ.get('OPENAI_API_KEY')

In [3]:
mmlu   = load_dataset("kz-transformers/mmlu-translated-kk")
const  = load_dataset("kz-transformers/kazakh-constitution-mc")
dastur = load_dataset("kz-transformers/kazakh-dastur-mc")
ent    = load_dataset("kz-transformers/kazakh-unified-national-testing-mc")
gsm    = load_dataset("kz-transformers/gsm8k-kk-translated")

In [7]:
import json


def format_mmlu_like(entry, dataset_name, index):
    title = entry.get("Title", "")
    text = entry.get("Text", "")
    question = entry.get("Question", "")
    options = []
    for opt_key in ["Option A", "Option B", "Option C", "Option D"]:
        if opt_key in entry and entry[opt_key] is not None:
            options.append(entry[opt_key])
            
    user_prompt = f"Title: {title}\nContext: {text}\nQuestion: {question}\nOptions:\n"
    for i, opt in enumerate(options):
        label = chr(ord('A') + i)
        user_prompt += f"{label}: {opt}\n"
    user_prompt += "\nБір ғана дұрыс жауаптың әріпін көрсетіңіз:"
    
    json_line = {
        "custom_id": f"{dataset_name}-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "system", 
                    "content": "Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан мәдениеті, ғылым, тарих және басқа да салалар бойынша көптаңдаулы сұрақтарға жауап беріңіз. Сұрақты және берілген жауап нұсқаларын мұқият оқып, ең дұрысын бір ғана әріппен (A, B, C, т.б.) белгілеңіз."
                },
                {"role": "user", "content": user_prompt}
            ],
            "max_tokens": 10
        }
    }
    return json_line


def format_ent_like(entry, dataset_name, subject_name, index):
    subject = entry.get("subject", "")
    question = entry.get("question", "")
    
    option_keys = [k for k in ['A','B','C','D','E','F','G','H'] if k in entry and entry[k] is not None and entry[k] != '<missing>']
    
    user_prompt = f"Subject: {subject}\nQuestion: {question}\nOptions:\n"
    for key in option_keys:
        user_prompt += f"{key}: {entry[key]}\n"
    user_prompt += "\nБір ғана дұрыс жауаптың әріпін көрсетіңіз:"
    
    json_line = {
        "custom_id": f"{dataset_name}-{subject_name}-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "messages": [
                {
                    "role": "system", 
                    "content": "Сіз қазақ тілінде жауап беретін білімді, пайдалы көмекшісіз. Қазақстан мәдениеті, ғылым, тарих және басқа да салалар бойынша көптаңдаулы сұрақтарға жауап беріңіз. Сұрақты және берілген жауап нұсқаларын мұқият оқып, ең дұрысын бір ғана әріппен (A, B, C, т.б.) белгілеңіз."
                },
                {"role": "user", "content": user_prompt}
            ],
            "max_tokens": 10
        }
    }
    return json_line


with open("batch_requests.jsonl", "w", encoding="utf-8") as f:
    ds = mmlu["validation"]
    for i, entry in enumerate(ds):
        line = format_mmlu_like(entry, "mmlu", i)
        f.write(json.dumps(line, ensure_ascii=False) + "\n")

    for dataset_name, dataset_obj in [("const", const), ("dastur", dastur)]:
        ds = dataset_obj["test"]
        for i, entry in enumerate(ds):
            line = format_mmlu_like(entry, dataset_name, i)
            f.write(json.dumps(line, ensure_ascii=False) + "\n")

    for subject_name in ent.keys():
        ds = ent[subject_name]
        for i, entry in enumerate(ds):
            line = format_ent_like(entry, "ent", subject_name, i)
            f.write(json.dumps(line, ensure_ascii=False) + "\n")

In [9]:
from openai import OpenAI
client = OpenAI(api_key=openai_key)

batch_input_file = client.files.create(
    file=open("batch_requests.jsonl", "rb"),
    purpose="batch"
)

In [11]:
batch_input_file_id = batch_input_file.id

client.batches.create(
  input_file_id=batch_input_file_id,
  endpoint="/v1/chat/completions",
  completion_window="24h",
  metadata={
    "description": "nightly eval job"
  }
)

Batch(id='batch_675b43f4cbb88190b501e849947a30cc', completion_window='24h', created_at=1734034420, endpoint='/v1/chat/completions', input_file_id='file-S5xZ6TSDMM2SPV6u15GHdK', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1734120820, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [53]:
batch_input_file

FileObject(id='file-S5xZ6TSDMM2SPV6u15GHdK', bytes=22905609, created_at=1734034155, filename='batch_requests.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [22]:
print(client.batches.retrieve('batch_675b43f4cbb88190b501e849947a30cc'))

Batch(id='batch_675b43f4cbb88190b501e849947a30cc', completion_window='24h', created_at=1734034420, endpoint='/v1/chat/completions', input_file_id='file-S5xZ6TSDMM2SPV6u15GHdK', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1734038114, error_file_id=None, errors=None, expired_at=None, expires_at=1734120820, failed_at=None, finalizing_at=1734036994, in_progress_at=1734034425, metadata={'description': 'nightly eval job'}, output_file_id='file-FQdij62Y18TyFwjwMVbXyN', request_counts=BatchRequestCounts(completed=17800, failed=0, total=17800))


In [61]:
file_response = client.files.content('file-FQdij62Y18TyFwjwMVbXyN')

In [62]:
response = file_response.text

In [67]:
with open('response.jsonl', 'w+') as f:
    f.write(response)

In [68]:
x = pd.read_json('response.jsonl', lines=True)